In [1]:
#MIRCV 2021
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#To support GPU
!pip install opencv-python==4.4.0.46
!pip install whoosh

     |████████████████████████████████| 49.5MB 87kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 471kB 4.1MB/s 


In [3]:
import cv2
import numpy as np
import os
import operator

from whoosh import index
from whoosh.fields import *
from whoosh.analysis import *
from whoosh import qparser

# for showing images in the cell outputs (Jupyter Notebooks / Google Colab)
from IPython.display import display
from ipywidgets import Image

from tqdm.notebook import tqdm

np.random.seed(42)  # makes the random pivot choice reproducible

BASE_DIR = '/content/gdrive/My Drive/mircv2021'
DEEP_PROTO = BASE_DIR + '/data/caffe/train_val.prototxt'
DEEP_MODEL = BASE_DIR + '/data/caffe/bvlc_reference_caffenet.caffemodel'
SRC_FOLDER = BASE_DIR + '/data/coco_img'
OUT_FOLDER = BASE_DIR + '/out'
WHOOSH_FOLDER = OUT_FOLDER + '/whoosh'
PIVOTS_FILE = OUT_FOLDER + '/pivots.txt.gz'
PIVOTS_ID_FILE = OUT_FOLDER + '/pivot_ids.txt'

IMAGE_ID_FILE = OUT_FOLDER + '/image_ids.txt'
FEATURES_FILE = OUT_FOLDER + '/extracted_features.txt.gz'

if not os.path.exists(OUT_FOLDER):
    os.mkdir(OUT_FOLDER)
if not os.path.exists(WHOOSH_FOLDER):
    os.mkdir(WHOOSH_FOLDER)

DEEP_LAYER = 'relu7'
SIZE = (227, 227)
MEAN_VALUES = (104, 117, 123)  # BGR

NUM_PIVOTS = 100
INDEX_FIELD = 'deep'
KX = 10
KQ = 10

K = 8

print("Variables Done")

Variables Done


In [4]:
class DNNExtractor:    
    
    def __init__(self, net_proto_path, trained_model_path, size, mean_values=None):
        self.size = size
        self.mean_values = mean_values

        self.net = cv2.dnn.readNetFromCaffe(net_proto_path, trained_model_path)
        # to enable GPU (this won't work on Colab without recompiling opencv)
        self.net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        self.net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
        
    
    def extract(self, img_file, layer, normalize=False):
        img = cv2.imread(img_file)
        blob = cv2.dnn.blobFromImage(img, 1.0, self.size, self.mean_values, swapRB=False, crop=False)
        self.net.setInput(blob)
        prob = self.net.forward(layer).flatten()

        if normalize:
            prob /= np.linalg.norm(prob)

        return prob
    
# it creates an instance of the DNNExtractor class
dnn = DNNExtractor(DEEP_PROTO, DEEP_MODEL, SIZE, MEAN_VALUES)

print("DNN Done")

DNN Done


In [19]:
def extract_features(img_folder):
    # get the list of image filenames
    filenames = [f for f in os.listdir(img_folder)]

    descriptors = [dnn.extract(os.path.join(SRC_FOLDER, f),
                               DEEP_LAYER, normalize=True) for f in tqdm(filenames)]

    descriptors = np.array(descriptors)
    filenames = np.array(filenames)
    
    return descriptors, filenames

# compute and cache features
if not os.path.exists(FEATURES_FILE):
  descriptors, ids = extract_features(SRC_FOLDER)
  np.savetxt(IMAGE_ID_FILE, ids, fmt='%s')
  np.savetxt(FEATURES_FILE, descriptors)

ids = np.loadtxt(IMAGE_ID_FILE, dtype='str')
descriptors = np.loadtxt(FEATURES_FILE)

print("Computing and cache features")

Computing and cache features


In [ ]:
#TODO: STEP 0

#make pivots
# get NUM_PIVOTS integers between 0 and len(descriptors)-1 without repetitions

# use the pivot array indices as pivot ids
if not os.path.exists(PIVOTS_FILE):

  pivots = None
  pivot_ids = None

  np.savetxt(PIVOTS_FILE, pivots)
  np.savetxt(PIVOTS_ID_FILE, pivot_ids)

In [ ]:
#TODO: STEP 1

pivots = np.loadtxt(PIVOTS_FILE)
pivot_ids = np.loadtxt(PIVOTS_ID_FILE)


def search(queryF, dataset, ids, k):
    cos_similarities = np.dot(dataset, queryF)
    result = zip(cos_similarities, ids)
    res = sorted(result, reverse=True)[:k]
    return res

#use pivots to transform features to Surrogare Text Representation (STR)
def features_2_text(img_f, top_k):
  #generate and return the STR of the img_f feature.
  #use the search function to retrieve the top_k nearest neighbor pivots

In [ ]:
#TODO: STEP 2

#Whoosh indexing
#Initialize the Whoosh index (see Exercise1)
#call features_2_text to transform the image features to STR
#index the STR of all images into the Whoosh index

In [ ]:
def display_image(filename, score=0):
    """ Displays an image and its corresponding score (optional)
        in Jupyter Notebook / Google Colab
    """
    filepath = os.path.join(SRC_FOLDER , filename)
    image = Image.from_file(filepath, width=300, height=400)
    print('{} - {:.3f}'.format(filename, score))  # :.3f = 3-decimal float
    display(image)


def display_results(results):
    """ Displays a set of results. Takes a list of (score, id) couples """
    for score, filename in results:
        display_image(filename, score)

In [ ]:
#TODO: STEP 3

#Whoosh searching
img_query = IMG_FOLDER + "/000000321557.jpg"
#Extract the feetures of img_query.
#Transform the features to STR
#Perform a Whoosh search and call display_results to show the search results

In [ ]:
# Optional reordering task

#get the IDs of images found by whoosh in hit
#transforms IDs into index positions in the results_descriptors
#to get the corresponding image features
#reorder the features using the search and
#show the result

display_results(reordered)